In [ ]:
from cracknuts.cracker import CrackerG1
from cracknuts.jupyter.cracker_g1_panel import CrackerG1Panel

In [ ]:
g1 = CrackerG1('192.168.0.19')

In [ ]:
g1.connect()

In [ ]:
CrackerG1Panel.builder()\
    .with_cracker(g1)\
    .with_acquisition(init=None, do=None, finish=None)\
    .build()